# EDA - Fraud Data

This notebook analyzes the `Fraud_Data.csv` dataset.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add src to path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data_loader import load_data
from preprocessing import clean_fraud_data

In [ ]:
# Load Data
df = load_data('../data/raw/Fraud_Data.csv')
df.head()

In [ ]:
# Basic Info
if df is not None:
    print(df.info())
    print(df.describe())

In [ ]:
# Class Imbalance
if df is not None and 'class' in df.columns:
    sns.countplot(x='class', data=df)
    plt.title('Class Distribution')
    plt.show()

In [ ]:
# Transaction Amount Distribution
if df is not None and 'purchase_value' in df.columns:
    sns.histplot(df['purchase_value'], bins=50)
    plt.title('Transaction Amount Distribution')
    plt.show()